### 🔬 SCIANTIX Output viewer

This notebook loads the `output.txt` file from SCIANTIX simulations and provides an interactive interface to:
- Browse available variables
- Select X and Y axes from the output columns
- Visualize any pair of variables using interactive Plotly charts


In [1]:
# !pip install pandas plotly ipywidgets
# !pip install --upgrade nbformat

import pandas as pd
import plotly.express as px
from ipywidgets import interact
import plotly.io as pio

# Ensure Plotly works in Jupyter notebooks
pio.renderers.default = 'notebook'

# Load the output file
df = pd.read_csv("output.txt", sep=r"\t+", engine="python")

# Show the first few rows
df.head()

,Time (h),Temperature (K),Fission rate (fiss / m3 s),Hydrostatic stress (MPa),Grain radius (m),Xe produced (at/m3),Xe in grain (at/m3),Xe in intragranular solution (at/m3),Xe in intragranular bubbles (at/m3),Xe at grain boundary (at/m3),...,Intergranular bubble radius (m),Intergranular bubble area (m2),Intergranular bubble volume (m3),Intergranular fractional coverage (/),Intergranular saturation fractional coverage (/),Intergranular gas swelling (/),Intergranular fractional intactness (/),Burnup (MWd/kgUO2),U235 (at/m3),U238 (at/m3)
0,0,1273,1.000000e+19,0,0.000005,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.5,0.000000,1,0.000000,7.209709e+26,2.301694e+28
1,55,1273,1.000000e+19,0,0.000005,4.752000e+23,4.490564e+23,4.490564e+23,0.000000e+00,2.614360e+22,...,8.318091e-09,1.275572e-16,4.064853e-25,0.002546,0.5,0.000005,1,0.067193,7.209709e+26,2.301694e+28
2,110,1273,1.000000e+19,0,0.000005,9.504000e+23,8.923260e+23,6.411603e+23,2.511657e+23,5.807396e+22,...,1.166885e-08,2.510230e-16,1.122167e-24,0.004996,0.5,0.000013,1,0.134386,7.209709e+26,2.301694e+28
3,165,1273,1.000000e+19,0,0.000005,1.425600e+24,1.327414e+24,9.502471e+23,3.771673e+23,9.818556e+22,...,1.470241e-08,3.985061e-16,2.244604e-24,0.007899,0.5,0.000027,1,0.201579,7.209709e+26,2.301694e+28
4,220,1273,1.000000e+19,0,0.000005,1.900800e+24,1.757899e+24,1.170399e+24,5.874995e+23,1.429014e+23,...,1.745095e-08,5.614305e-16,3.753455e-24,0.011076,0.5,0.000044,1,0.268772,7.209709e+26,2.301694e+28


In [2]:
# List of columns from the file
columns = df.columns.tolist()

# Interactive plot using dropdowns
@interact(x=columns, y=columns)
def plot_sciantix(x='Time (h)', y='Fission gas release (/)'):
    """
    Plots an interactive line chart of the selected y-variable against x-variable
    from the SCIANTIX output dataframe.
    """
    fig = px.line(df, x=x, y=y, title=f"{y} vs {x}")
    fig.show()


interactive(children=(Dropdown(description='x', options=('Time (h)', 'Temperature (K)', 'Fission rate (fiss / …